<a href="https://colab.research.google.com/github/sastrawanikadek/bangkit-assignment-5-face-recognition-CNN/blob/master/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>